# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
# print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkifydb 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)


#### Set Keyspace

In [7]:
try:
    session.set_keyspace('sparkifydb')
except Exception as e:
    print(e)


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
create_table1 = 'CREATE TABLE IF NOT EXISTS music_library (sessionId int, itemInSession int, artist text, songTitle text, songLength decimal, PRIMARY KEY(sessionId, itemInSession))'

try:
    session.execute(create_table1)
except Exception as e:
    print(e)



In [9]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_library(sessionId, itemInSession, artist, songTitle, songLength) "
        query = query + "VALUES(%s, %s, %s, %s, %s)"
        
        query_data =(int(line[8]), int(line[3]), line[0], line[9], float(line[5])) # sessionId, itemInSession, artist, songTitle, songLength
        
        # load data into table
        try:
            session.execute(query, query_data)
        except Exception as e:
            print(e)
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
#         session.execute(query, (line[#], line[#]))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
query = 'SELECT * from music_library WHERE sessionId = 338 AND itemInSession = 4'
result_set = session.execute(query)

for result in result_set:
    print(result)

Row(sessionid=338, iteminsession=4, artist='Faithless', songlength=Decimal('495.3073'), songtitle='Music Matters (Mark Knight Dub)')


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [11]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

create_table2 = 'CREATE TABLE IF NOT EXISTS artist_library (artist text, song_title text, itemInSession int, user_first_name text, user_last_name text, userId int, sessionId int, PRIMARY KEY(userId, sessionId))'

try:
    session.execute(create_table2)
except Exception as e:
    print(e)   

In [12]:
# Insert data
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO artist_library(artist, song_title, itemInSession, user_first_name, user_last_name, userId, sessionId) "
        query = query + "VALUES(%s, %s, %s, %s, %s, %s, %s)"
        
        query_data =(line[0], line[9], int(line[3]), line[1], line[4], int(line[10]), int(line[3])) # artist, song_title, itemInSession, user_first_name, user_last_name, userId, sessionId
        
        # load data into table
        try:
            session.execute(query, query_data)
        except Exception as e:
            print(e)

In [13]:
# Confirm data insertion
query = 'SELECT * from artist_library WHERE userId = 10'
result_set = session.execute(query)

for result in result_set:
    print(result)

Row(userid=10, sessionid=0, artist='Brad Paisley With Andy Griffith', iteminsession=0, song_title="Waitin' On A Woman", user_first_name='Sylvie', user_last_name='Cruz')
Row(userid=10, sessionid=1, artist='Klaus Badelt', iteminsession=1, song_title='Moonlight Serenade', user_first_name='Sylvie', user_last_name='Cruz')
Row(userid=10, sessionid=2, artist='Eminem', iteminsession=2, song_title='Criminal', user_first_name='Sylvie', user_last_name='Cruz')
Row(userid=10, sessionid=3, artist='Laurel & Hardy', iteminsession=3, song_title="At the Ball_ That's All", user_first_name='Sylvie', user_last_name='Cruz')
Row(userid=10, sessionid=4, artist='Percubaba', iteminsession=4, song_title='Intro', user_first_name='Sylvie', user_last_name='Cruz')
Row(userid=10, sessionid=5, artist='Rilo Kiley', iteminsession=5, song_title='A Man/Me/Then Jim (Album Version)', user_first_name='Sylvie', user_last_name='Cruz')
Row(userid=10, sessionid=6, artist='Watain', iteminsession=6, song_title='Sworn To The Dark',

In [14]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
create_table3 = 'CREATE TABLE IF NOT EXISTS user_library (userId int, sessionId int, user_first_name text, user_last_name text, song_title text, PRIMARY KEY(song_title, userId, sessionId))'

try:
    session.execute(create_table3)
except Exception as e:
    print(e)



                    

In [15]:
# Insert data
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        try:
            query = "INSERT INTO user_library(userId, sessionId, user_first_name, user_last_name, song_title) "
            query = query + "VALUES(%s, %s, %s, %s, %s)"
            query_data = int(line[10]), int(line[8]), line[1], line[4], line[9] # userId, sessionId, firstName, lastName, song
        except Psycopg2.Error as e:
            print(e)
            
        # load data into table
        try:
            session.execute(query, query_data)
        except Exception as e:
            print(e)

In [16]:
# Confirm data insertion
query = "SELECT user_first_name, user_last_name FROM user_library WHERE song_title='You Gotta Be' AND userId = 8"
result_set = session.execute(query)

for result in result_set:
    print(result)

Row(user_first_name='Kaylee', user_last_name='Summers')


### Drop the tables before closing out the sessions

In [17]:
query1 = 'DROP TABLE user_library';
query2 = 'DROP TABLE artist_library';
query3 = 'DROP TABLE music_library';

query_list = [query1, query2, query3]

for query in query_list:
    session.execute(query)


### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()